# <center> POVM as a quantum version of the classifier </center>

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
#dev = qml.device('default.qubit', wires=2)

In [3]:
#dev = qml.device('qiskit.aer', wires=2)
dev = qml.device('qiskit.ibmq', wires=2, shots=8192, ibmqx_token="e942e97ce86ca8c3609a4053fe6762ec3db17c41895b2d85d6a3560a1156501d57db36753f99138ecb32dca58a97ae5b38005ad39855dd92ab0e86cef852c1a2")

## A module of the two-element povm.

In [4]:
def two_element_povm(params, wire0, wire1):
    #params 

    # Controlled-RY gate controlled by first qubit in |0> state
    qml.PauliX(wires=wire0)
    qml.CRY(params[0], wires=[wire0,wire1])
    qml.PauliX(wires=wire0)
    
    # Controlled-RY gate controlled by first qubit in |1> state
    qml.CRY(params[1], wires=[wire0,wire1])

    # Controlled-Rotation gate (arbitrary single-qubit unitary operator) controlled by 2nd qubit in |0> state
    qml.PauliX(wires=wire1)
    qml.CRot(params[2], params[3], params[4], wires=[wire1,wire0])
    qml.PauliX(wires=wire1)

    # # Controlled-Rotation gate (arbitrary single-qubit unitary operator) controlled by 2nd qubit in |1> state
    qml.CRot(params[5], params[6], params[7], wires=[wire1,wire0])

Drawing `two_element_povm` module

In [4]:
@qml.qnode(dev)
def povm_circuit(params):
    two_element_povm(params, 0, 1)
    return qml.expval(qml.Identity(0) @ qml.PauliZ(1))

# initial parameters
params = np.random.random([8])
povm_circuit(params)

print("Drawing a two-element POVM circuit : ")
print(povm_circuit.draw())

Drawing a two-element POVM circuit : 
 0: ──X──╭C──────────X──╭C──────────RZ(-0.0555)──╭X──RZ(-0.119)──RY(-0.255)──╭X──RY(0.255)──RZ(0.174)──RZ(-0.0588)──╭X──RZ(-0.327)──RY(-0.213)──╭X──RY(0.213)──RZ(0.386)──╭┤ ⟨I ⊗ Z⟩ 
 1: ─────╰RY(0.122)─────╰RY(0.992)──X────────────╰C──────────────────────────╰C──X──────────────────────────────────╰C──────────────────────────╰C────────────────────────╰┤ ⟨I ⊗ Z⟩ 



---

## Preparing two arbitrary pure states

In [5]:
def state_preparation(params):
    return

---

## Minimum Error Discrimination

Preparation: $\{q_i,\hat{\rho}_i\}^{N}_{i=1}$ $\longleftarrow$ $\hat{\rho} = \sum_i q_i\hat{\rho}_i$. <br>

POVMs: $\{E_k\}_{k=1}^{L}$ with $\sum_{k=1}^{L} E_k = I$.

Q. Why "$N=L$" is optimal?

In general, for a state $\hat{\rho}_i$ generated in preparation, a detection event on $\hat{E}_k$ happens with probability $ p(k|i) = tr [\hat{E}_k\hat{\rho_i}] $.

$$
p_{success} = \sum_i q_i p(i|i)
$$
$$
p_{guess} = \max_{E} \sum_i q_i p(i|i)
$$
$$
p_{error} = 1 - p_{guess}
$$

For two-element POVMs $\{\hat{E}_0, \hat{E}_1\}$ on two different state prepared as $\{(q_0,\hat{\rho}_0), (q_1,\hat{\rho}_1)\}$, 
$$
p_{error} = 1 - p_{guess} = 1-\max_{\vec{\theta}}\left(q_0\text{Tr}[\hat{E}_0\hat{\rho}_0] + q_1\text{Tr}[\hat{E}_1\hat{\rho}_1] \right)
$$

---

## Classification of two different states by using POVM

$|\Psi^0_i\rangle := |\psi_0\rangle|0\rangle$ 
$\longrightarrow$ POVM $\longrightarrow$ 
$|\Psi^0_f\rangle := \left(\hat{K}_0|\psi_0\rangle\right)|0\rangle + \left(\hat{K}_1|\psi_0\rangle\right)|1\rangle$

$$
\text{Tr}[\hat{\rho}_0\hat{E}_0] = \langle\psi_0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle \langle 0 | 0 \rangle = \langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle\\
=\frac{1}{2}\left(\langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle + \langle\psi_0|\langle 1|\hat{K}^\dagger_1\hat{K}_1|\psi_0\rangle|1\rangle\right)\\
+\frac{1}{2}\left(\langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle - \langle\psi_0|\langle 1|\hat{K}^\dagger_1\hat{K}_1|\psi_0\rangle|1\rangle\right) \\ 
\frac{1}{2} \left(1 + \langle\Psi_f|I\otimes Z|\Psi_f\rangle\right)
$$
Similarly, $\text{Tr}[\hat{\rho}_0\hat{E}_1] = \frac{1}{2}\left(1 - \langle\Psi_f|I\otimes Z|\Psi_f\rangle\right)$.

Find **the optimal POVMs** for discriminating two different states:
$$
p_{error} = 1- (-1)\min_{\vec{\theta}}\left(-q_o\text{Tr}[\hat{E}_0\hat{\rho}_0] - q_1\text{Tr}[\hat{E}_1\hat{\rho}_1]\right)\\
=\min_{\vec{\theta}}\left[\frac{1}{2} - \frac{q_0}{2}\langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle + \frac{q_1}{2}\langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle\right]
$$

$ \langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle $

In [6]:
@qml.qnode(dev, diff_method="parameter-shift")
def circuit_povm_expvalIZpsi0(params):
    # Initial state: |0>
    qml.Hadamard(wires=0) # |+>

    # arbitrary rotation
    qml.Rot(params[0], params[1], params[2], wires=0)

    # two-element POVM
    two_element_povm(params[3:], 0, 1)

    return qml.expval(qml.Identity(0) @ qml.PauliZ(1)) 

$ \langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle $

In [7]:
@qml.qnode(dev, diff_method="parameter-shift")
def circuit_povm_expvalIZpsi1(params):
    # Initial state: X|0> = |1>
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0) # |->

    # arbitrary rotation
    qml.Rot(params[0], params[1], params[2], wires=0)

    # two-element POVM
    two_element_povm(params[3:], 0, 1)

    return qml.expval(qml.Identity(0) @ qml.PauliZ(1))

Cost function : $C(\vec{\theta}) = \frac{1}{2}\left(1 - q_0\langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle + q_1\langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle\right)$

In [8]:
def cost(x):
    K0psi0 = circuit_povm_expvalIZpsi0(x)
    K1psi1 = circuit_povm_expvalIZpsi1(x)
    return (1/2) * (1 - (1/2)*K0psi0 + (1/2)*K1psi1)


### Optimization of POVM: solving $\min_{\vec{\theta}}C(\vec{\theta})$

In [9]:
grad_function = qml.grad(cost)

In [11]:
params = 2 * np.pi * np.random.random([11])
print(grad_function(params))

[ 0.07406616 -0.00134277  0.00094604 -0.17501461  0.00992295  0.00021362
 -0.00138855  0.00198364 -0.00018311 -0.00036621  0.00161743]


In [11]:
print("Drawing a circuit of povm_expvalIZ_psi0 : ")
print(circuit_povm_expvalIZpsi0.draw())
print("Drawing a circuit of povm_expvalIZ_psi1 : ")
print(circuit_povm_expvalIZpsi1.draw())

Drawing a circuit of povm_expvalIZ_psi0 : 
 0: ──H──Rot(3.14, 1.57, 2.79)──X──╭C─────────X──╭C────────────╭Rot(1.4, 2.91, 1.88)─────╭Rot(0.674, 3.61, 0.156)──╭┤ ⟨I ⊗ Z⟩ 
 1: ───────────────────────────────╰RY(6.28)─────╰RY(3.14)──X──╰C─────────────────────X──╰C────────────────────────╰┤ ⟨I ⊗ Z⟩ 

Drawing a circuit of povm_expvalIZ_psi1 : 
 0: ──X──H──Rot(3.14, 1.57, 2.79)──X──╭C─────────X──╭C────────────╭Rot(1.4, 2.91, 1.88)─────╭Rot(0.674, 3.61, 0.156)──╭┤ ⟨I ⊗ Z⟩ 
 1: ──────────────────────────────────╰RY(6.28)─────╰RY(3.14)──X──╰C─────────────────────X──╰C────────────────────────╰┤ ⟨I ⊗ Z⟩ 



## Calculating Kraus Operators or POVMs

### Arbitrary rotation of the single qubit
### $$ U = R_z(\beta)R_y(\gamma)R_z(\delta) = \begin{bmatrix} e^{-i(\delta+\beta)/2}\cos(\gamma/2) & -e^{i(\delta-\beta)/2}\sin(\gamma/2) \\ e^{-i(\delta-\beta)/2}\sin(\gamma/2) & e^{i(\delta+\beta)/2}\cos(\gamma/2) \end{bmatrix}$$

### Y-Rotation of the single qubit
### $$ R_y(\phi) = e^{-i\phi\sigma_y/2} = \begin{bmatrix} \cos\phi/2 & -\sin\phi/2 \\ \sin\phi/2 & \cos\phi/2 \end{bmatrix} $$

In [12]:
def unitaries_in_povm(params):
    U = qml.Rot(params[0], params[1], params[2], wires=2).matrix
    Ry0 = qml.RY(params[3], wires=2).matrix
    Ry1 = qml.RY(params[4], wires=2).matrix
    V0 = qml.Rot(params[5], params[6], params[7], wires=2).matrix
    V1 = qml.Rot(params[8], params[9], params[10], wires=2).matrix

    return U, Ry0, Ry1, V0, V1

U, Ry0, Ry1, V0, V1 = unitaries_in_povm(params)

$$
\hat{K}_0 = V_0D_0U \\
\hat{K}_1 = V_1D_1U 
$$

In [13]:
def kraus_op(params):
    U, _, _, V0, V1 = unitaries_in_povm(params)
    D0 = np.diag([np.cos(params[3]/2), np.cos(params[4]/2)])
    D1 = np.diag([np.sin(params[3]/2), np.sin(params[4]/2)])
    K0 = np.dot(np.dot(V0, D0), U)
    K1 = np.dot(np.dot(V1, D1), U)

    return K0, K1

K0, K1 = kraus_op(params)

In [35]:
print("K0 =", K0)
print("K1 =", K1)

K0 = [[0.00856563-0.0820617j  0.00856972-0.08206349j]
 [0.64334034+0.28161269j 0.64334029+0.28161217j]]
K1 = [[-0.68536188-0.05876237j  0.68536251+0.05876222j]
 [-0.15901747-0.03923677j  0.15901473+0.039237j  ]]


$ \hat{K}^\dagger_1\hat{K}_0 + \hat{K}^\dagger_2\hat{K}_1 = I$

In [14]:
np.dot(K0.conj().T, K0) + np.dot(K1.conj().T, K1)

tensor([[1.+0.00000000e+00j, 0.-1.38777878e-17j],
        [0.+1.38777878e-17j, 1.+0.00000000e+00j]], requires_grad=True)

$ \langle \psi_0 |\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle$

In [15]:
res = np.dot(K0, np.array([1/np.sqrt(2),1/np.sqrt(2)]))
np.dot(res.conj(),res)

(0.9999999999959829+0j)

$ \langle \psi_1 |\hat{K}^\dagger_1\hat{K}_1|\psi_1\rangle$

In [16]:
res = np.dot(K1, np.array([1/np.sqrt(2),-1/np.sqrt(2)]))
np.dot(res.conj(),res)

(0.9999999999898925+0j)